# make_geojson.ipynb

### 概要

宇都宮市の[オープンソースライブラリ](https://data.city.utsunomiya.tochigi.jp/)から、[防災データ](https://catalog.city.utsunomiya.tochigi.jp/dataset/shiteikinkyuuhinanbashoichiran)を用いて公開するgeojsonファイルを作成します

In [1]:
# 一時避難場所(令和3年8月現在)
tmp_evacuation_path='https://www.city.utsunomiya.tochigi.jp/_res/projects/default_project/_page_/001/009/937/ichizihinanbasyo0608.csv'
# 広域避難場所（令和３年８月現在）
wide_area_evacuation_path='https://www.city.utsunomiya.tochigi.jp/_res/projects/default_project/_page_/001/009/938/kouikihinanbasyo.csv'

In [2]:
import pandas as pd # データフレーム
import requests # csvへのリクエスト
import io # ファイル名の管理

#### csvデータの読み取り

In [3]:
# 広域避難場所
r_wide = requests.get(wide_area_evacuation_path)
df_wide = pd.read_csv(io.BytesIO(r_wide.content),sep=",",encoding='shift-jis')

In [4]:
# 一時避難場所
r_tmp = requests.get(tmp_evacuation_path)
df_tmp = pd.read_csv(io.BytesIO(r_tmp.content),sep=",",encoding='shift-jis')

In [5]:
tmp_park_df = df_tmp[df_tmp['名称'].str.contains('公園')]
tmp_park_df

,名称,住所,電話番号,座標
9,錦中央公園（一時避難場所）,今泉町157,028-663-1611（スポーツ振興財団）,139.9024669:36.5657864
11,御幸公園（一時避難場所）,平出工業団地13-1,028-663-1611（スポーツ振興財団）,139.9337652:36.5803203
24,文化会館及び明保野公園（一時避難場所）,明保野町7-66 外,028-636-2121（文化会館）,139.8713327:36.5436572
29,陽南第1公園（一時避難場所）,大塚町1,028-663-1611（スポーツ振興財団）,139.8656817:36.5321054
44,宇都宮城址公園（一時避難場所）,本丸町、旭1丁目地内（市役所東側）,028-632-2529（市公園管理課）,139.8886906:36.5513311


In [6]:
wide_park_df = df_wide[df_wide['名称'].str.contains('公園')]
wide_park_df

,名称,住所,電話番号,座標
1,ブレックスアリーナ宇都宮〔市体育館〕・駅東公園（広域避難場所）,元今泉5丁目6-18 他,028-663-1611（スポーツ振興財団）,139.9127068:36.5581267
3,八幡山公園（広域避難場所）,塙田５丁目2-70,028-632-2529（市公園管理課）,139.8899724:36.5676559
4,平出工業団地公園（広域避難場所）,平出工業団地30-1,028-662-5813（緑の相談所）,139.9389889:36.5677532
6,宮原運動公園（広域避難場所）,陽南４丁目5-6,028-658-1052,139.8717313:36.5350165
7,栃木県中央公園（広域避難場所）,睦町2-50,028-636-1491（公園管理事務所）,139.8663676:36.5553989


#### geojsonの作成

In [11]:
from geojson import Point, Feature, FeatureCollection
import folium
from folium.plugins import MarkerCluster # 地図を出力するためのモジュール

In [10]:
latitude = '35.584047' 
longitude = '139.665936'
altitude = '14.000000'
# my_point = Point((float(longitude), float(latitude), float(altitude)))
my_point = Point((float(longitude), float(latitude)))
my_feature = Feature(geometry=my_point)
my_feature_collection = FeatureCollection(my_feature)

print (my_feature_collection)

{"features": {"geometry": {"coordinates": [139.665936, 35.584047], "type": "Point"}, "properties": {}, "type": "Feature"}, "type": "FeatureCollection"}


In [30]:
def visualize_locations(
                        # df, # データフレーム
                        # longitude_column_name, # データフレームの中で緯度を保存したカラム名
                        # latitude_column_name, # データフレームの中で経度を保存したカラム名
                        zoom=15, # ズームの大きさ、デフォルトで4
                        # idx_range = 15000 # 出力する行数の上限
                        ) -> folium.folium.Map: # Foliumのマップ型を返す
    """
    EXPLAIN : 日本を拡大した地図に、pandasデータフレームのlatitudeおよびlongitudeカラムをプロットする。
    ARGS :
    % df, # データフレーム
    % longitude_column_name, # データフレームの中で緯度を保存したカラム名
    % latitude_column_name, # データフレームの中で経度を保存したカラム名
    % zoom=4, # ズームの大きさ、デフォルトで4
    idx_range = 15000 # 出力する行数の上限
    """

    # 図の大きさを指定する。
    f = folium.Figure(width=1000, height=500)

    # 初期表示の中心の座標を指定して地図を作成する。
    #北緯36度33分55.6秒 東経139度53分0.7秒
    center_lat=36.5590232
    center_lon=139.8966108
    # foliumのMap形のコンストラクタに初期の座標とズーム情報を渡す, fは図の大きさとして指定
    m = folium.Map([center_lat,center_lon], zoom_start=zoom).add_to(f)
#     marker_cluster = MarkerCluster().add_to(m)
    # データフレームの全ての行のマーカーを作成する。
    # 総データ数の10分の1だけのデータを扱う、つまり1万3千程度、これ以上だと出力できない事態が発生する。
    # if df.shape[0] < idx_range : # 出力上限よりデータフレームの行数が小さい場合はデータフレームの行数分だけ出力する
    #   for idx in range(df.shape[0]):
    #     folium.Marker(location=[df[longitude_column_name][idx], df[latitude_column_name][idx]]).add_to(marker_cluster)
    # else:
    #   for idx in range(idx_range): # データフレームの行数が出力上限以上の場合は出力上限分だけ出力する
    #     folium.Marker(location=[df[longitude_column_name][idx], df[latitude_column_name][idx]]).add_to(marker_cluster)

    return m

In [31]:
visualize_locations()